# NetPyNE Tutorial 9: Evolutionary Algorithm for Parameters Optimization

In [1]:
!pip install inspyred


In [2]:
import os
# if not os.path.exists('nb2_files/batch_evol'):
#     os.makedirs('nb2_files/batch_evol')
os.chdir(os.getcwd() + '/nb2_files/batch_evol')
print(os.getcwd())

/mnt/disk15tb/adam/git_workspace/netpyne/hdmea_simulations/BurstingPlotDevelopment/nb2_files/batch_evol




Two examples are provided: 'simple' and 'complex'.
In 'simple', 3 parameters are optimized to match target firing rates in 2 populations.
In 'complex', 6 parameters are optimized to match target firing rates in 6 populations.

First we import all necessary modules, and then describe parameters space to explore.

In [3]:
# %matplotlib inline
from netpyne import specs
from netpyne.batch import Batch

params = specs.ODict()
params['probEall'] = [0.05, 0.2] # 0.1
params['weightEall'] = [0.0025, 0.0075] #5.0
params['probIE'] = [0.2, 0.6] #0.4
params['weightIE'] = [0.0005, 0.002]
params['probLengthConst'] = [100,200]
params['stimWeight'] = [0.05, 0.2]

pops = {}
pops['E2'] = {'target': 5, 'width': 2, 'min': 1}
pops['I2'] = {'target': 10, 'width': 5, 'min': 2}
pops['E4'] = {'target': 30, 'width': 10, 'min': 1}
pops['I4'] = {'target': 10, 'width': 3, 'min': 2}
pops['E5'] = {'target': 40, 'width': 4, 'min': 1}
pops['I5'] = {'target': 25, 'width': 5, 'min': 2}

Provide fitness function and it's arguments:

In [4]:
fitnessFuncArgs = {}
fitnessFuncArgs['pops'] = pops
fitnessFuncArgs['maxFitness'] = 1000

def fitnessFunc(simData, **kwargs):
    import numpy as np
    pops = kwargs['pops']
    maxFitness = kwargs['maxFitness']
    popFitness = [None for i in pops.items()]
    popFitness = [min(np.exp(  abs(v['target'] - simData['popRates'][k])  /  v['width']), maxFitness)
            if simData["popRates"][k]>v['min'] else maxFitness for k,v in pops.items()]
    fitness = np.mean(popFitness)
    popInfo = '; '.join(['%s rate=%.1f fit=%1.f'%(p,r,f) for p,r,f in zip(list(simData['popRates'].keys()), list(simData['popRates'].values()), popFitness)])
    print('  '+popInfo)
    return fitness

Here we create `Batch` object with paramaters to tune, and set output folder, optimization method ('evol'), fitness function and all the configurations from above.

In [6]:
batch = Batch(params=params)

# Set output folder, grid method (all param combinations), and run configuration
batch.batchLabel = 'complex'
batch.saveFolder = './' + batch.batchLabel
batch.method = 'evol'
batch.runCfg = {
	'type': 'mpi_bulletin',#'hpc_slurm',
	'script': 'init.py',
	# options required only for hpc
	'mpiCommand': 'mpirun',
	'nodes': 1,
	'coresPerNode': 2,
	'allocation': 'default',
	'email': 'amwe@ucdavis.edu',
	'reservation': None,
	#'folder': '/home/salvadord/evol'
	#'custom': 'export LD_LIBRARY_PATH="$HOME/.openmpi/lib"' # only for conda users
}
batch.evolCfg = {
	'evolAlgorithm': 'custom',
	'fitnessFunc': fitnessFunc, # fitness expression (should read simData)
	'fitnessFuncArgs': fitnessFuncArgs,
	'pop_size': 6,
	'num_elites': 1, # keep this number of parents for next generation if they are fitter than children
	'mutation_rate': 0.4,
	'crossover': 0.5,
	'maximize': False, # maximize fitness function?
	'max_generations': 4,
	'time_sleep': 5, # wait this time before checking again if sim is completed (for each generation)
	'maxiter_wait': 40, # max number of times to check if sim is completed (for each generation)
	'defaultFitness': 1000 # set fitness value in case simulation time is over
}

Now we can run this code. When completed, you can inspect folder `tut_batch_evol/{networkType}`, where `{networkType}` is either `simple` or `complex`, to find the output of each generation and the summary for all, including the optimal values of requested parameters (short summary will also appear in the output of the cell). 

Note that the algorithm is stochastic, so you will not get the same results if re-run it again.

In [8]:
batch.run()

Saving batch to ./complex/complex_batch.json ... 
['probEall', 'weightEall', 'probIE', 'weightIE', 'probLengthConst', 'stimWeight'] [0.12962380919275168, 0.0064387127763064385, 0.5352322758834878, 0.0012324885065375774, 148.9491237916065, 0.15995889134204477]
set probEall=0.12962380919275168
set weightEall=0.0064387127763064385
set probIE=0.5352322758834878
set weightIE=0.0012324885065375774
set probLengthConst=148.9491237916065
set stimWeight=0.15995889134204477
Saving simConfig to ./complex/gen_0/gen_0_cand_0_cfg.json ... 
--------------------------------------------------------------------------------
['probEall', 'weightEall', 'probIE', 'weightIE', 'probLengthConst', 'stimWeight'] [0.14073352009734763, 0.003692229763531849, 0.4686334154946002, 0.000728383830027002, 197.34190214408324, 0.061331777157537286]
set probEall=0.14073352009734763
set weightEall=0.003692229763531849
set probIE=0.4686334154946002
set weightIE=0.000728383830027002
set probLengthConst=197.34190214408324
set st

KeyboardInterrupt: 